# Clasificador - hito 2

In [1]:
import pandas as pd
import numpy as np
import pickle
from string import punctuation
 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})
import plotly
import plotly.express as px

file_names = {
    "df_es_mapping": "../../Data/mapping/df_es_mapping.pickle",
    "df_us_mapping": "../../Data/mapping/df_us_mapping.pickle",
    
    "df_es_test": "../../Data/test/df_es_test.pickle",
    "df_us_test": "../../Data/test/df_us_test.pickle",
    
    "df_es_train": "../../Data/train/df_es_train.pickle",
    "df_us_train": "../../Data/train/df_us_train.pickle",
    
    "df_es_trial": "../../Data/trial/df_es_trial.pickle",
    "df_us_trial": "../../Data/trial/df_us_trial.pickle",
}

# mas imports

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer # tokenizer especial para tweets
tt = TweetTokenizer()
# nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation


**cargar sets**

In [2]:
df_es_train = pickle.load(open(file_names["df_es_train"], "rb"))
df_es_trial = pickle.load(open(file_names["df_es_trial"], "rb"))
df_es_test = pickle.load(open(file_names["df_es_test"], "rb"))

df_us_train = pickle.load(open(file_names["df_us_train"], "rb"))
df_us_trial = pickle.load(open(file_names["df_us_trial"], "rb"))
df_us_test = pickle.load(open(file_names["df_us_test"], "rb"))

**pre-procesamiento**

In [3]:
df_us_train['tokenized_text'] = df_us_train['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))
df_us_train.head()

,id,text,label,tokenized_text
0,729044324441186304,Selfies for summatime @ Drexel University,12,selfies for summatime @ drexel university
1,663834134037442560,Ready to be a bulldog with rasso #hailstate #i...,14,ready to be a bulldog with rasso #hailstate #i...
2,747449193350963200,#scored my new #matcotools #slidehammer weight...,16,#scored my new #matcotools #slidehammer weight...
3,691439672761925637,@user last night was so much fun @ Skyway Thea...,6,@user last night was so much fun @ skyway theatre
4,758118895618109440,love beach days @ Manasquan Beach,12,love beach days @ manasquan beach


In [4]:
df_us_test['tokenized_text'] = df_us_test['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))

In [5]:
"""stopwords_en_withpunct = set(stopwords_en).union(set(punctuation))
print(list(stopwords_en_withpunct)[:10])""";

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vectorizer = CountVectorizer(min_df=5)
X_train_bow = vectorizer.fit_transform(df_us_train["tokenized_text"])
X_test_bow = vectorizer.transform(df_us_test["tokenized_text"])

In [8]:
from sklearn.naive_bayes import MultinomialNB

In [9]:
clf = MultinomialNB()
clf.fit(X_train_bow, df_us_train["label"])

MultinomialNB()

In [10]:
clf.score(X_train_bow, df_us_train["label"])

0.39701310639001064

In [11]:
from sklearn.metrics import classification_report

In [12]:
df_us_mapping = pickle.load(open(file_names["df_us_mapping"], "rb")).sort_values("label")
df_us_mapping

,label,emoji,name
0,0,❤,_red_heart_
1,1,😍,_smiling_face_with_hearteyes_
10,10,📷,_camera_
11,11,🇺🇸,_United_States_
12,12,☀,_sun_
13,13,💜,_purple_heart_
14,14,😉,_winking_face_
15,15,💯,_hundred_points_
16,16,😁,_beaming_face_with_smiling_eyes_
17,17,🎄,_Christmas_tree_


In [13]:
y_pred = clf.predict(X_test_bow)
print(classification_report(df_us_test["label"], y_pred, target_names=df_us_mapping["emoji"]))

              precision    recall  f1-score   support

           ❤       0.35      0.58      0.44     10798
           😍       0.25      0.25      0.25      4830
           📷       0.16      0.16      0.16      1432
          🇺🇸       0.47      0.50      0.48      1949
           ☀       0.25      0.43      0.32      1265
           💜       0.32      0.05      0.08      1114
           😉       0.12      0.04      0.06      1306
           💯       0.27      0.14      0.19      1244
           😁       0.14      0.03      0.05      1153
           🎄       0.60      0.60      0.60      1545
           📸       0.29      0.10      0.15      2417
           😜       0.04      0.01      0.01      1010
           😂       0.30      0.52      0.38      4534
           💕       0.19      0.05      0.08      2605
           🔥       0.45      0.47      0.46      3716
           😊       0.09      0.06      0.07      1613
           😎       0.16      0.11      0.13      1996
           ✨       0.29    

In [14]:
vocab = {k: v for v, k in enumerate(vectorizer.get_feature_names_out())}

In [15]:
vec_test = np.zeros(X_train_bow.shape[1])
k = vocab["santa"]
vec_test[k] = 1
print(vectorizer.inverse_transform([vec_test])[0][0])
clf.predict_proba([vec_test])

santa


array([[0.21267139, 0.10559105, 0.03170409, 0.02263593, 0.06533366,
        0.01011697, 0.02992065, 0.00932074, 0.02055197, 0.11856118,
        0.02276163, 0.02159743, 0.10988494, 0.03433423, 0.01711626,
        0.03922043, 0.05487754, 0.02576475, 0.02707579, 0.02095938]])

### Top palabras por emoji

In [16]:
%%time
vocab_length = X_train_bow.shape[1]
proba_matrix = np.array([clf.predict_proba(np.eye(1,vocab_length,k))[0] for k in range(vocab_length)])

CPU times: user 1min 20s, sys: 999 ms, total: 1min 21s
Wall time: 13.6 s


In [17]:
print(vocab_length)
print(proba_matrix.shape)

29983
(29983, 20)


In [18]:
una_linea = proba_matrix[:,3]
una_linea.shape

(29983,)

In [19]:
def topPalabras(proba_matrix,emoji_id,k=5):
    # retorna las palabras para las cuales el emoji en cuestión tiene mas probabilidad
    prob = proba_matrix[:,emoji_id]  # mmm
    ind = np.argpartition(prob,-k)[-k:]
    val = prob[ind]
    palabras = [vectorizer.inverse_transform([np.eye(1,vocab_length,k)[0]])[0][0] for k in ind]
    return palabras, val

In [20]:
i = 3
map_emojis = [0,1,10,11,12,13,14,15,16,17,18,19,2,3,4,5,6,7,8,9]
print(df_us_mapping["emoji"][map_emojis[i]])
topPalabras(proba_matrix,i)

🇺🇸


(['murica', 'imwithher', 'election2016', 'ivoted', 'merica'],
 array([0.78007421, 0.8062308 , 0.83135636, 0.86388407, 0.8821097 ]))

In [21]:
for i in range(20):
    print(df_us_mapping["emoji"][map_emojis[i]])
    pal, val = topPalabras(proba_matrix,i)
    print(dict([(pal[j],val[j]) for j in range(len(pal))]))

❤
{'heart': 0.5761150508287712, 'valentines': 0.5811796285055364, 'lovemyfamily': 0.5793072574283205, 'valentine': 0.5949253260518091, 'loveofmylife': 0.6651429760457039}
😍
{'inlove': 0.4288496011644503, 'gorg': 0.4453369489332203, 'obsessed': 0.44883209548006375, 'swoon': 0.45600734120528374, 'swooning': 0.4562752924146042}
📷
{'sony': 0.3882939691961041, 'gdlfashion': 0.4052054276523981, 'bvillain': 0.5117380913776236, 'shredforaliving': 0.42318456197855087, 'kae': 0.4156412082407767}
🇺🇸
{'murica': 0.7800742137338681, 'imwithher': 0.8062308003990193, 'election2016': 0.8313563588022287, 'ivoted': 0.8638840691974496, 'merica': 0.8821097001791353}
☀
{'soakin': 0.447527739339606, 'sun': 0.45989390878761877, 'sunny': 0.4740267973356278, 'sunshine': 0.5826999203405687, 'beachin': 0.4927352286841576}
💜
{'snyder': 0.3559341490500346, 'ripprince': 0.3575182172656461, 'endalz': 0.4398786494003444, 'purple': 0.5506017875276537, 'purplerain': 0.5134869871082338}
😉
{'backtowork': 0.212917247503517